In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob

# submission = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')


In [ ]:
tr = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
max(tr[tr.u_out==0]['pressure']),min(tr[tr.u_out==0]['pressure'])

In [ ]:
test_pred = pd.concat([pd.read_csv(i) for i in glob.glob('../input/pulp-fiction-fold5-fold*/submission.csv')],1)

In [ ]:
test_pred.to_csv('submission_all.csv',index=False)

In [ ]:
test_preds = [test_pred[f'pressure_{i}'].values for i in range(5)]

In [ ]:
class config:
    paths = {
        # data
        'train': '../input/ventilator-pressure-prediction/train.csv',
        'test' : '../input/ventilator-pressure-prediction/test.csv',
        'ss'   : '../input/ventilator-pressure-prediction/sample_submission.csv',    
    }
    
    model_params = {
        'is_train': True,
        'debug'   : False,
        
        'EPOCH'   : 300,
        'BATCH_SIZE' : 1024,
        'NUM_FOLDS' : 10,
    }
    
    post_processing = {
        'max_pressure': 64.82099173863948,
        'min_pressure': -1.8957442945646408,
        'diff_pressure': 0.07030214545121617,
    }

In [ ]:
def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val


In [ ]:
submission["pressure"] = np.median(np.vstack(test_preds), axis=0)
# submission["pressure"] = np.round((submission.pressure - config.post_processing["min_pressure"])/config.post_processing["diff_pressure"]) * config.post_processing["diff_pressure"] + config.post_processing["min_pressure"]
# submission.pressure = np.clip(submission.pressure, config.post_processing["min_pressure"], config.post_processing["max_pressure"])
# display(submission.head())

In [ ]:
unique_pressures = submission["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)
submission["pressure"] = submission["pressure"].apply(find_nearest)

submission.to_csv('submission_base.csv', index=False)